## LAVA JATO - AN INFOGRAPHIC OF THE CORRUPTION SCANDALS IN BRAZIL

In [130]:
from os import listdir
from os.path import isfile, join

delacoes = ['./Delações/' + f for f in listdir('./Delações/') if isfile(join('./Delações/', f))]

delacoes[:10]



['./Delações/Colaboração premiada - Alberto Youssef - Depoimento 1.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 10.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 11.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 12.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 13.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 14.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 15.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 16.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 17.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 18.pdf']

In [131]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFPage
from pdfminer.pdfparser import *
from io import StringIO

txtDelacoes = []

def convert_pdf_to_txt(path):
    retstr = StringIO()
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, retstr, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    fp = open(delacoes[0], 'rb')

    parser = PDFParser(fp)
    document = PDFDocument()
    parser.set_document(document)
    document.set_parser(parser)

    for page in document.get_pages():
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

txtDelacoes.append(convert_pdf_to_txt(delacoes[0]))

print(txtDelacoes[0][:300])

--------------------------------~-~ -

CONFIDENCIAL 
POLICIA FEDERAL 

SUPERINTEND~NCIA REGIONAL NO ESTAOO 00 PARANÁ 

TERMO DE COLABORAÇÃO N ° 01 

TERMO DE DECLARA Ç ÕES 

que presta ALBERTO YOUSSEF 

DRCOR - Delegacia Regional de Combate ao Crime Organizado 
DELEFIN - Delegacia de Repressão a C r


In [132]:
import re

def fixOcr(pattern):
    result = ""
    for c in pattern:
        result += c + "\s?"
        
    result = re.sub('[DO0]','[DO0]',result)
    result = re.sub('Ê','[Ê~]',result)
    result = re.sub('Á','[AÁ]',result)
    return result

def extractDelation(delationPdfText):
    delationPdfText = delationPdfText[re.search('\nque presta ',delationPdfText).span()[0]:]
    delationPdfText = re.sub(fixOcr('CONFIDENCIAL \nPOLICIA FEDERAL'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('\nCONFIDENCIAL'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('\nSUPERINTENDIONCIA'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('SUPERINTENDÊNCIA REGIONAL NO ESTAOO DO PARANÁ'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('REGIONAL NO ESTAOO DO PARANÁ'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE COLABORAÇÃO ') + 'N[^\n]+','',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE DECLARAÇÕES '),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DRCOR - Delegacia Regional de Comba') + '[tl]' + fixOcr('e ao Crime Organizado'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE DECLARAÇÕES'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DELEFIN - Delegacia de Repressão a Crimes contra o Sistema Financeiro e Desvio de Verbas Públicas'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DELEFIN - Delegacia de Repressão a Crimes contra o Sistema Financeiro e Desvio de Veribas Públicas'),'',delationPdfText)
    delationPdfText = delationPdfText[:re.search('\nDECLARANTE:',delationPdfText).span()[0]]
    delationPdfText = re.sub('\n',' ',delationPdfText)
    return delationPdfText
    
result = extractDelation(txtDelacoes[0])

result[:500]

' que presta ALBERTO YOUSSEF      16402272884 Pet 5245  Ao(s) 02 dia(s) do mês de outubro de 2014, nesta Superintendência Regional do  Departamento de Polícía Federal, em Curitiba/PR, perante EDUARDO MAUAT DA SILVA,  Delegada de Polícia Federal, Classe Especial, matrícula nO 8190, atendendo a requisição  do Procurador Geral da República constante do Ofício n° 1152/Gab para se proceder á  oitiva de ALBERTO YOUSSEF, brasileiro, casado, RG 3506470-21PR, CPF 532.050.659- 72, filho de Kalim Youssef e '

In [134]:
words = str.split(result, ' ')

wordCount = {}

for word in words:
    if(re.match('\d', word) == None and len(word) > 1):
        fixedWord = str.lower(re.sub('[,;\."]$','',word))
        fixedWord = re.sub('^"','',fixedWord)
        if(fixedWord in wordCount):
            wordCount[fixedWord] += 1
        else:
            wordCount[fixedWord] = 1

mostCommonUselessWords = ['-', '', 'de','era', 'eram','junto','a', 'é','•', 'feito','que','diz','anteriormente','serem',
                          'fazer','tendo','quem','dos', 'segundo','ter','sendo','art.', 'conforme','qual','afirma','o', 
                          'pelas','podendo','quais','do', 'acerca','da', 'fim','no', 'em', 'se', 'como', 'um', 'uma', 
                          'os', 'as', 'para', 'e', 'por', 'na', 'com', 'Dos', 'ou', 'das', 'são', 'ao', 'entre', 'mais', 
                          'nos', 'ser', 'sua', 'mas', 'tem', 'também', 'pela', 'sobre', 'seu', 'pelo', 'pode', 'termos',
                          'assim', 'esta', 'nas', 'já', 'aos', 'questão', 'diferentes', 'termo', 'in', 'mesmo', 'século',
                          'outro', 'este', 'isso', 'pois', 'há', 'isto', 'lugar', 'the', 'esse', 'outros', 'sem', 'uso', 
                          'essa', 'lá', 'foi', 'ainda', 'ele', 'não', 'só', 'realização','determinada','algum','seria',
                          'assevera','pessoa','teve','próprio','ate','inicialmente','sempre','vezes','havia','pet','passou',
                          'presente','algumas','\x0c', 'período','recorda-se','antes','nota','prévio','seja','sido',
                          'todavia','feitos','inclusive','meio','teve','sido','declarante']

for uselessWord in mostCommonUselessWords:
    wordCount.pop(uselessWord, None)
    
sortedWordCount = sorted(wordCount.items(), key=lambda word: word[1])
sortedWordCount.reverse()
sortedWordCount[:25]

[('valores', 21),
 ('janene', 20),
 ('empreiteiras', 20),
 ('empresas', 20),
 ('paulo', 20),
 ('roberto', 17),
 ('contratos', 11),
 ('petrobras', 10),
 ('costa', 10),
 ('jose', 10),
 ('caso', 10),
 ('empresa', 10),
 ('questionado', 9),
 ('ano', 9),
 ('diretoria', 8),
 ('pagamentos', 7),
 ('esquema', 7),
 ('valor', 7),
 ('federal', 6),
 ('lei', 6),
 ('emissão', 6),
 ('relação', 6),
 ('grandes', 5),
 ('serviços', 5),
 ('espécie', 5)]